# Script pré-traitement des données apprentissage du thème sur les colonnes titre et description (suppression de tous les doublons)

## Modules à importer

In [1]:
import pandas as pd
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

## Lecture du fichier CSV et création de la dataframe

In [2]:
opinion_df = pd.read_csv ("./data_website_tab.csv", sep="\t", names=["titre", "description","position","theme"],dtype={"titre":"object","description":"object","position":"category","theme":"category"},header=None)

## Suppression des lignes vides

In [3]:
opinion_df_clean = opinion_df.dropna(how='any')

## Récupération des index des lignes sans texte ne contenant que des espaces

In [4]:
opinion_df_spaces = opinion_df_clean[(opinion_df_clean.description == " ") |(opinion_df_clean.description == "  ")]

## Récupération des index des lignes sans texte ne contenant que des espaces

In [5]:
opinion_df_clean2 = opinion_df_clean.drop(index=opinion_df_spaces.index)

## Remplacement du jeu de données initial par le jeu de données néttoyé

In [6]:
opinion_df= opinion_df_clean2

# Suppression des doublons

## Nombre de lignes pour chaque thème avant suppression des doublons

In [7]:
opinion_df.theme.value_counts()

elevage                    2214
chasse                     1765
zoo-cirque                  748
mer-pisciculture            712
animaux-de-compagnie        687
droit-animal                653
corrida                     511
experimentation-animale     268
Name: theme, dtype: int64

# Nombre total de thèmes avant suppression

In [8]:
opinion_df.theme.count()

7558

In [9]:
opinion_df.head(30)

,titre,description,position,theme
0,Le député Loïc Dombreval demande au gouverneme...,.@LoicDombreval se prononce pour une anticipa...,Agit pour les animaux,elevage
1,Ces maires ont attribué une délégation conditi...,Ces maires ont attribué à un conseiller ou un...,Agit pour les animaux,droit-animal
2,Tribune en faveur de la censure du débat publi...,"Soutien assumé à la convention « Déméter », d...",Agit contre les animaux,elevage
3,Proposition de loi n°1896 visant à abolir l’él...,"En France, il est interdit de fabriquer et ve...",Agit pour les animaux,elevage
4,3 Députés demandent au gouvernement d'intégrer...,Question écrite de M. Dimitri Houbron député ...,Agit pour les animaux,mer-pisciculture
5,3 Députés demandent au gouvernement d'intégrer...,Question écrite de M. Dimitri Houbron député ...,Agit pour les animaux,elevage
6,Le sénateur Arnaud Bazin demande des explicati...,Question écrite n° 18999 de M. Arnaud Bazin ...,Agit pour les animaux,experimentation-animale
7,La ville de Montpellier vote un vœu visant à i...,Le Conseil municipal de la ville de Montpelli...,Agit pour les animaux,zoo-cirque
8,"Éric Piolle, maire de Grenoble, soutient le Ré...",Dans une vidéo publiée sur le compte Twitter ...,Agit pour les animaux,droit-animal
9,"Anne Hidalgo, maire de Paris, et Christophe Na...",À l'occasion de la journée mondiale des anima...,Agit pour les animaux,droit-animal


# Suppression totale de tout texte dupliqué de les colonnes titre et description 

In [10]:
opinion_df_clean3 = opinion_df.drop_duplicates(subset=['titre', 'description'], keep=False)

In [11]:
opinion_df_clean3.head(30)

,titre,description,position,theme
0,Le député Loïc Dombreval demande au gouverneme...,.@LoicDombreval se prononce pour une anticipa...,Agit pour les animaux,elevage
1,Ces maires ont attribué une délégation conditi...,Ces maires ont attribué à un conseiller ou un...,Agit pour les animaux,droit-animal
2,Tribune en faveur de la censure du débat publi...,"Soutien assumé à la convention « Déméter », d...",Agit contre les animaux,elevage
3,Proposition de loi n°1896 visant à abolir l’él...,"En France, il est interdit de fabriquer et ve...",Agit pour les animaux,elevage
6,Le sénateur Arnaud Bazin demande des explicati...,Question écrite n° 18999 de M. Arnaud Bazin ...,Agit pour les animaux,experimentation-animale
7,La ville de Montpellier vote un vœu visant à i...,Le Conseil municipal de la ville de Montpelli...,Agit pour les animaux,zoo-cirque
8,"Éric Piolle, maire de Grenoble, soutient le Ré...",Dans une vidéo publiée sur le compte Twitter ...,Agit pour les animaux,droit-animal
9,"Anne Hidalgo, maire de Paris, et Christophe Na...",À l'occasion de la journée mondiale des anima...,Agit pour les animaux,droit-animal
10,Tours baptise un jardin du nom de l'éléphant F...,Extrait du site de la ville de Tours : Homma...,Agit pour les animaux,droit-animal
11,La ville de Pessac adopte une motion «en faveu...,CONSEIL MUNICIPAL DU MARDI 29 SEPTEMBRE 2020 ...,Agit pour les animaux,droit-animal


# Nombre de lignes pour chaque thème après suppression des doublons

In [12]:
 opinion_df_clean3.theme.value_counts()

elevage                    1719
chasse                     1528
zoo-cirque                  714
animaux-de-compagnie        522
corrida                     467
droit-animal                445
mer-pisciculture            354
experimentation-animale     239
Name: theme, dtype: int64

# Nombre total de thèmes après suppression

In [13]:
 opinion_df_clean3.theme.count()

5988

# Remplacement des index

In [14]:
opinion_df_clean3 = opinion_df_clean3.reset_index(drop=True)

# Remplacement du jeu de données initial par le jeu de données nettoyé

In [15]:
opinion_df = opinion_df_clean3

In [16]:
opinion_df.head()

,titre,description,position,theme
0,Le député Loïc Dombreval demande au gouverneme...,.@LoicDombreval se prononce pour une anticipa...,Agit pour les animaux,elevage
1,Ces maires ont attribué une délégation conditi...,Ces maires ont attribué à un conseiller ou un...,Agit pour les animaux,droit-animal
2,Tribune en faveur de la censure du débat publi...,"Soutien assumé à la convention « Déméter », d...",Agit contre les animaux,elevage
3,Proposition de loi n°1896 visant à abolir l’él...,"En France, il est interdit de fabriquer et ve...",Agit pour les animaux,elevage
4,Le sénateur Arnaud Bazin demande des explicati...,Question écrite n° 18999 de M. Arnaud Bazin ...,Agit pour les animaux,experimentation-animale


# Extraction pour thème

In [17]:
X = opinion_df[["titre", "description"]]
y = opinion_df.theme

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Création des différentes fonctions utilisées

In [19]:
class SingleColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, data_dict):
        return data_dict[self.key]
    
class MultiColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns].to_dict('records')
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

In [20]:
def split_into_tokens_nltk(desc) :
    return word_tokenize(desc)

In [21]:
fs = FrenchStemmer()
def split_into_stems(desc):
    words = word_tokenize(desc)
    return [fs.stem(w) for w in words]

## Liste des nombres et lois + signes de ponctuation

In [22]:
stopwords = open('stopwords_list.txt','r').read().split()
nblaw_stopwords = stopwords+list(string.punctuation)

## Pipeline pour la colonne titre + transformation du texte en sac de mots avec CountVectorizer

In [23]:
var_vectorizer = CountVectorizer(tokenizer=split_into_stems, lowercase=True, stop_words=nblaw_stopwords, min_df=0.01)

title_pipeline = make_pipeline(
    SingleColumnSelector(key="titre"),
    var_vectorizer
)

## Pipeline pour la colonne description + transformation du texte en vecteurs avec TfidfVectorizer

In [24]:
desc_vectorizer = TfidfVectorizer(tokenizer=split_into_stems,lowercase=True, stop_words=nblaw_stopwords, min_df=0.01)

description_pipeline = make_pipeline(
    SingleColumnSelector(key="description"),
    desc_vectorizer
)

## Union des pipelines de pré-traitement

In [25]:
union = FeatureUnion(transformer_list= [
    ("description_features", description_pipeline),
    ("titre_features", title_pipeline)
])

preprocess_pipeline = make_pipeline(
        union
)